This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [2]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'JqMz3qbywvFF6Kux6FGB'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [36]:
# First, import the relevant modules
import requests
import collections
import pandas as pd
import datetime as dt

In [31]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
resp = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X")
json = resp.json()

## Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
respjson['dataset']['data'][0][1]

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [138]:
## 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
## 2. Convert the returned JSON object into a Python dictionary.
df = pd.json_normalize(json['dataset'], 'data', ['date', 'number', 'name'], 
                    record_prefix='data_',errors='ignore')
df = df.iloc[:,0:7]
df.columns = ('Date','Open','High','Low','Close','Change','Volume')
df['Date'] = pd.to_datetime(df['Date'])
df = df[df['Date'].dt.year  == 2017]
df.head()

,Date,Open,High,Low,Close,Change,Volume
727,2017-12-29,51.76,51.94,51.45,51.76,NaN,34640.0
728,2017-12-28,51.65,51.82,51.43,51.60,NaN,40660.0
729,2017-12-27,51.45,51.89,50.76,51.82,NaN,57452.0
730,2017-12-22,51.05,51.50,50.92,51.32,NaN,71165.0
731,2017-12-21,51.16,51.52,50.90,51.40,NaN,120649.0


In [123]:
## 3. Calculate what the highest and lowest opening prices were for the stock in this period.
highest = df.loc[df['Open'] == df['Open'].max()]
lowest = df.loc[df['Open'] == df['Open'].min()]
print('Highest Opening Day: \n'+ str(highest) + '\n')
print('Lowest Opening Day: \n'+ str(lowest))

Highest Opening Day: 
          Date   Open   High    Low  Close  Change    Volume
736 2017-12-14  53.11  53.54  52.15  52.67     NaN  132981.0

Lowest Opening Day: 
          Date  Open   High    Low  Close  Change   Volume
965 2017-01-24  34.0  34.35  33.85  34.22     NaN  48797.0


In [134]:
## 4. What was the largest change in any one day (based on High and Low price)?
high_low_diff = df.loc[df['High'] - df['Low'] == (df['High'] - df['Low']).max()]
high_low_diff

,Date,Open,High,Low,Close,Change,Volume,day_change
888,2017-05-11,43.4,46.06,43.25,45.0,NaN,189125.0,0.022227


In [139]:
## 5. What was the largest change between any two days (based on Closing Price)?
df['% Day Change'] = (df['Close'].pct_change(1))*100
df.loc[df['% Day Change'] == df['% Day Change'].max()]

,Date,Open,High,Low,Close,Change,Volume,% Day Change
826,2017-08-08,44.9,45.09,44.15,44.37,NaN,156168.0,6.122937


In [140]:
## 6. What was the average daily trading volume during this year?
df['Volume'].mean()

89124.33725490196

In [141]:
## 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
df['Volume'].median()

76286.0